In [2]:
# setting the environment variables
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from config import set_environment
set_environment()

In [3]:
from langchain_community.document_loaders import DocusaurusLoader


import nest_asyncio

nest_asyncio.apply()

loader = DocusaurusLoader("https://python.langchain.com")
docs = loader.load()
loader = DocusaurusLoader(
    "https://python.langchain.com",
    filter_urls=[
        "https://python.langchain.com/docs/integrations/document_loaders/sitemap"
    ],
)
documents = loader.load()
documents[0]

USER_AGENT environment variable not set, consider setting it to identify your requests.
Error fetching https://python.langchain.com/docs/integrations/chat/perplexity/ with attempt 1/3: Cannot connect to host python.langchain.com:443 ssl:default [Temporary failure in name resolution]. Retrying...
Error fetching https://python.langchain.com/docs/integrations/chat/pipeshift/ with attempt 1/3: Cannot connect to host python.langchain.com:443 ssl:default [Temporary failure in name resolution]. Retrying...
Error fetching https://python.langchain.com/docs/integrations/document_transformers/ai21_semantic_text_splitter/ with attempt 1/3: Cannot connect to host python.langchain.com:443 ssl:default [Temporary failure in name resolution]. Retrying...
Error fetching https://python.langchain.com/docs/integrations/document_transformers/beautiful_soup/ with attempt 1/3: Cannot connect to host python.langchain.com:443 ssl:default [Temporary failure in name resolution]. Retrying...
Error fetching https:/

Document(metadata={'source': 'https://python.langchain.com/docs/integrations/document_loaders/sitemap/', 'loc': 'https://python.langchain.com/docs/integrations/document_loaders/sitemap/', 'changefreq': 'weekly', 'priority': '0.5'}, page_content='\n\n\n\n\nSitemap | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchProvidersAnthropicAWSGoogleHugging FaceMicrosoftOpenAIMoreProvidersAbsoAcreomActiveloop Deep LakeAerospikeAI21 LabsAimAINetworkAirbyteAirtableAlchemyAleph AlphaAlibaba CloudAnalyticDBAnnoyAnthropicAnyscaleApache Software FoundationApache DorisApifyAppleArangoDBArceeArcGISArgillaArizeArthurArxivAscendAskNewsAssemblyAIAstra DBAtlasAwaDBAWSAZLyricsBAAIBagelBagelDBBaichuanBaiduBananaBasetenBeamBeautiful SoupBibTeXBiliBiliBittensorBlackboardbookend.aiBoxBrave Searc

In [1]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

underlying_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
)
# Avoiding unnecessary costs by caching the embeddings.
embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [ ]:
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name="gemini-pro")

In [ ]:
from langchain import hub

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")